In [1]:
import pandas as pd
import  csv 
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

import numpy as np
import seaborn as sns


Modeling PCE

In [2]:

#OLS 
precovid_stationary_file="df_precovid.csv"
postcovid_stationary_file="df_postcovid.csv"

stationary_file ="df_stationary.csv"

df_precovid=pd.read_csv(precovid_stationary_file, index_col=0)
df_postcovid=pd.read_csv(postcovid_stationary_file, index_col=0)
df_stationary=pd.read_csv(stationary_file, index_col=0)


In [4]:
# how to concatenate two dataframes 
df_normal= pd.concat([df_precovid, df_postcovid])


In [5]:
df_normal

,UNRATE,PCE,CPI,FEDFUNDS,M2,WAGE_AVG,INDPRO,GDPDEF,CSENT,CRUDE,COMPI,LABOR,break_1,break_2
2006-05-31,-0.1,0.004364,0.600,0.15,0.001013,-0.000993,0.000251,0.000,-8.3,4.260482,0.009088,0.0,0,0
2006-06-30,0.0,0.003007,0.500,0.05,0.005591,0.004955,0.003362,0.000,5.8,4.261988,0.001808,0.1,0,0
2006-07-31,0.1,0.009052,1.100,0.25,0.006082,0.002962,-0.000653,0.586,-0.2,4.309604,0.004205,-0.1,0,0
2006-08-31,0.0,-0.000171,0.900,0.01,0.004544,0.001477,0.004331,0.000,-2.7,4.291049,0.006573,0.1,0,0
2006-09-30,-0.2,0.003547,-1.000,0.00,0.003934,0.003929,-0.001813,0.000,3.4,4.155722,-0.015002,-0.1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-30,0.1,0.004322,0.885,-0.19,0.004721,0.003657,-0.002581,0.000,1.3,4.247781,0.000514,0.0,1,1
2024-12-31,-0.1,0.009832,1.154,-0.16,0.000807,0.001964,0.010677,0.000,2.2,4.250181,0.000837,0.0,1,1
2025-01-31,-0.1,-0.002543,1.483,-0.15,0.003217,0.004195,-0.001595,1.127,-2.3,4.327339,0.015416,0.1,1,1
2025-02-28,0.1,0.002811,0.689,0.00,0.003402,0.001952,0.009577,0.000,-7.0,4.270161,0.008273,-0.2,1,1


In [6]:
y=df_normal['PCE']
# get all factors from the csv 
# Filter for rows where target == 'PCE' and select predictor and lag
precovid_df=pd.read_csv("granger_outputs/precovid_significant_factors.csv", index_col=0)
postcovid_df=pd.read_csv("granger_outputs/postcovid_significant_factors.csv", index_col=0)
precovid_factors = precovid_df.loc[precovid_df['target'] == 'PCE', ['predictor', 'lag']].astype({'lag': int})
postcovid_factors = postcovid_df.loc[postcovid_df['target'] == 'PCE', ['predictor', 'lag']].astype({'lag': int})

print(precovid_factors)
print(postcovid_factors)


   predictor  lag
6      COMPI    1
7        CPI    1
8      CRUDE    3
9   FEDFUNDS    6
10    INDPRO    3
11        M2    5
12    UNRATE    1
  predictor  lag
3     COMPI    1
4    GDPDEF    4
5        M2    1
6    UNRATE    1
7  WAGE_AVG    3


In [7]:
predictor={
    'COMPI':1,
    'CPI': 1,
    'CRUDE':3,
    'FEDFUNDS':6,
    'INDPRO':3,
    'M2':5,
    'UNRATE':1

}
predictor2={
    'COMPI':1,
    'GDPDEF':4,
    'M2':1,
    'UNRATE':1,
    'WAGE_AVG':3,
}


In [8]:
import statsmodels.api as sm

In [9]:
df_lagged=pd.DataFrame()
df_lagged['PCE']=df_normal['PCE']
def OLS_results(factors, df):
    df=df.copy()
    for pred, lag in factors.values:
        df[f'{pred}_lag{lag}']= df_normal[pred].shift(lag)

    df= df.dropna()

    y= df['PCE']
    X= df[ [col for col in df.columns if col !='PCE'] ]
    # Add constant for intercept
    X = sm.add_constant(X)

    # Fit OLS model
    model = sm.OLS(y, X).fit()

    # Print summary
    print(model.summary())

OLS_results(precovid_factors, df_lagged)


                            OLS Regression Results                            
Dep. Variable:                    PCE   R-squared:                       0.263
Model:                            OLS   Adj. R-squared:                  0.237
Method:                 Least Squares   F-statistic:                     10.12
Date:                Thu, 23 Oct 2025   Prob (F-statistic):           8.30e-11
Time:                        17:44:09   Log-Likelihood:                 889.56
No. Observations:                 206   AIC:                            -1763.
Df Residuals:                     198   BIC:                            -1737.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0102      0.004      2.854

In [10]:
OLS_results(postcovid_factors, df_lagged)

                            OLS Regression Results                            
Dep. Variable:                    PCE   R-squared:                       0.176
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     8.613
Date:                Thu, 23 Oct 2025   Prob (F-statistic):           2.04e-07
Time:                        17:44:10   Log-Likelihood:                 887.41
No. Observations:                 208   AIC:                            -1763.
Df Residuals:                     202   BIC:                            -1743.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0026      0.001      4.541

In [11]:
refined_precovid_factors= (precovid_factors.loc[~precovid_factors['predictor'].isin(['CPI', 'M2'])])
refined_postcovid_factors= (postcovid_factors.loc[~postcovid_factors['predictor'].isin(['GDPDEF', 'M2', 'WAGE_AVG'])])

In [12]:
OLS_results(refined_precovid_factors, df_lagged)

                            OLS Regression Results                            
Dep. Variable:                    PCE   R-squared:                       0.250
Model:                            OLS   Adj. R-squared:                  0.232
Method:                 Least Squares   F-statistic:                     13.36
Date:                Thu, 23 Oct 2025   Prob (F-statistic):           3.07e-11
Time:                        17:44:12   Log-Likelihood:                 887.74
No. Observations:                 206   AIC:                            -1763.
Df Residuals:                     200   BIC:                            -1744.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0114      0.003      3.277

In [13]:
OLS_results(refined_postcovid_factors, df_lagged)

                            OLS Regression Results                            
Dep. Variable:                    PCE   R-squared:                       0.163
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     20.19
Date:                Thu, 23 Oct 2025   Prob (F-statistic):           9.67e-09
Time:                        17:44:13   Log-Likelihood:                 898.45
No. Observations:                 211   AIC:                            -1791.
Df Residuals:                     208   BIC:                            -1781.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0033      0.000     13.597      

In [14]:
df_stationary.columns

Index(['UNRATE', 'PCE', 'CPI', 'FEDFUNDS', 'M2', 'WAGE_AVG', 'INDPRO',
       'GDPDEF', 'CSENT', 'CRUDE', 'COMPI', 'LABOR', 'break_1', 'break_2'],
      dtype='object')

In [15]:
type(df_stationary.index)
df_stationary.index=pd.to_datetime(df_stationary.index)

In [32]:

# cols = list(set(precovid_factors['predictor']).union(set(postcovid_factors['predictor'])) )+['PCE']
# cols = list(set(postcovid_factors['predictor']) )+['PCE']
# cols=['COMPI', 'GDPDEF', 'UNRATE','PCE']
cols=['COMPI','UNRATE','CRUDE', 'FEDFUNDS','INDPRO','PCE']
# ,PCE,COMPI,1,3.335206162856171e-05
# 7,PCE,CPI,1,0.0051443197413389
# 8,PCE,CRUDE,3,0.00020166334414702916
# 9,PCE,FEDFUNDS,6,0.000118916537661428
# 10,PCE,INDPRO
# cols= list(set(postcovid_factors['predictor']) )+['PCE']
# cols=list([col for col in  df_stationary.columns if col not in ['break_1', 'break_2']])
exog_vars=['break_1', 'break_2']
df_vars = df_stationary.copy()


In [33]:
df_vars.shape


(227, 14)

In [34]:
cols

['COMPI', 'UNRATE', 'CRUDE', 'FEDFUNDS', 'INDPRO', 'PCE']

In [35]:
print(df_vars.isna().sum())

UNRATE      0
PCE         0
CPI         0
FEDFUNDS    0
M2          0
WAGE_AVG    0
INDPRO      0
GDPDEF      0
CSENT       0
CRUDE       0
COMPI       0
LABOR       0
break_1     0
break_2     0
dtype: int64


In [36]:
#22 months  for test and holdout
df_vars.index= pd.to_datetime(df_vars.index)
split_idx1= int(0.9 * len(df_vars) )
split_idx2=int(0.95*len(df_vars))
train , val ,test= df_vars.iloc[:split_idx1],df_vars.iloc[split_idx1:split_idx2], df_vars.iloc[split_idx2:]


In [37]:
from sklearn.metrics import mean_squared_error

To do make a holdout set too on which we do this testing analysis , since doing on test is cheating

In [38]:
type(val.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [39]:
from statsmodels.tsa.api import VAR
import math

lags = range(1, 12)

forecast_dfs = {set_name: {} for set_name in ['val', 'test']}
rmse_results = {set_name: {} for set_name in ['val', 'test']}


for lag in lags:
    try:
        # Build and fit VAR with exogenous regressors
        model = VAR(endog=train[cols], exog=train[exog_vars])
        results = model.fit(maxlags=lag, trend='ctt')  # 'c' = constant only

        # Forecast validation
        forecast_val = results.forecast(
            y=train[cols].values[-results.k_ar:],
            steps=len(val),
            exog_future=val[exog_vars].values
        )

        # Forecast test
        forecast_test = results.forecast(
            y=train[cols].values[-results.k_ar:],
            steps=len(test),
            exog_future=test[exog_vars].values
        )

        # Create DataFrames
        forecast_val_df = pd.DataFrame(forecast_val, index=val.index, columns=cols)
        forecast_test_df = pd.DataFrame(forecast_test, index=test.index, columns=cols)

        # Compute RMSE for PCE
        rmse_val = np.sqrt(mean_squared_error(val["PCE"], forecast_val_df["PCE"]))
        rmse_test = np.sqrt(mean_squared_error(test["PCE"], forecast_test_df["PCE"]))

        # Store results
        rmse_results['val'][lag] = rmse_val
        rmse_results['test'][lag] = rmse_test
        forecast_dfs['val'][lag] = forecast_val_df
        forecast_dfs['test'][lag] = forecast_test_df

        print(f"Lag={lag} fitted successfully, val_RMSE={rmse_val:.4f}, test_RMSE={rmse_test:.4f}")

    except Exception as e:
        print(f"Failed at lag={lag}: {e}")

# Summarize RMSEs
rmse_df = pd.DataFrame(rmse_results)
print("\nRMSE Summary (rows=lags):")
print(rmse_df)



Lag=1 fitted successfully, val_RMSE=0.0031, test_RMSE=0.0039
Lag=2 fitted successfully, val_RMSE=0.0035, test_RMSE=0.0039
Lag=3 fitted successfully, val_RMSE=0.0038, test_RMSE=0.0041
Lag=4 fitted successfully, val_RMSE=0.0034, test_RMSE=0.0037
Lag=5 fitted successfully, val_RMSE=0.0042, test_RMSE=0.0042
Lag=6 fitted successfully, val_RMSE=0.0056, test_RMSE=0.0052
Lag=7 fitted successfully, val_RMSE=0.0052, test_RMSE=0.0055
Lag=8 fitted successfully, val_RMSE=0.0048, test_RMSE=0.0052
Lag=9 fitted successfully, val_RMSE=0.0043, test_RMSE=0.0057
Lag=10 fitted successfully, val_RMSE=0.0096, test_RMSE=0.0103
Lag=11 fitted successfully, val_RMSE=0.0084, test_RMSE=0.0081

RMSE Summary (rows=lags):
         val      test
1   0.003096  0.003892
2   0.003547  0.003919
3   0.003772  0.004082
4   0.003425  0.003715
5   0.004219  0.004172
6   0.005646  0.005241
7   0.005232  0.005497
8   0.004824  0.005186
9   0.004297  0.005714
10  0.009561  0.010254
11  0.008357  0.008092


/opt/homebrew/Caskroom/miniconda/base/envs/dsc_env/lib/python3.14/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)
/opt/homebrew/Caskroom/miniconda/base/envs/dsc_env/lib/python3.14/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)
/opt/homebrew/Caskroom/miniconda/base/envs/dsc_env/lib/python3.14/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)
/opt/homebrew/Caskroom/miniconda/base/envs/dsc_env/lib/python3.14/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)
/opt/homebrew/Caskroom/minic

In [40]:

set_name = 'val'   # change to 'test' for test plots

n_lags = len(forecast_dfs[set_name])
cols_grid = 3
rows_grid = math.ceil(n_lags / cols_grid)

fig, axes = plt.subplots(rows_grid, cols_grid, figsize=(18, 5 * rows_grid), sharex=True, sharey=True)
axes = axes.flatten()

for i, (lag, df) in enumerate(forecast_dfs[set_name].items()):
    ax = axes[i]
    ax.plot(val.index, val["PCE"], label="Actual", linewidth=2, color='black')
    ax.plot(val.index, df["PCE"], '--', label="Forecast")
    ax.set_title(f"Lag={lag}, RMSE={rmse_results[set_name][lag]:.3f}")
    ax.legend()

# Remove empty axes
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.xlabel("Date")
plt.ylabel("PCE")
plt.suptitle(f"PCE Forecast vs Actual using VAR ({set_name.upper()} set)", fontsize=16)
plt.tight_layout()
plt.savefig("forecast_grid.png", dpi=150, bbox_inches='tight')
plt.close()

In [41]:


# We choose lag 9
forecast_test_df = forecast_dfs['test'][9]  # log-diff forecast

forecast_test_df.index = pd.to_datetime(forecast_test_df.index)
df_all = pd.read_csv("df_all.csv", index_col=0)
df_all.index = pd.to_datetime(df_all.index)

train_all, val_all, test_all = df_all.iloc[:split_idx1], df_all.iloc[split_idx1:split_idx2], df_all.iloc[split_idx2:]

# Last log value from validation set
last_log = np.log(val_all['PCE'].iloc[-1])

# Back-transform log-diffs to levels
forecast_log_levels = last_log + forecast_test_df['PCE'].cumsum().shift(fill_value=0)
df_forecast_level = pd.DataFrame(index=forecast_test_df.index)
df_forecast_level['PCE'] = np.exp(forecast_log_levels)  # convert to levels

# Align test to forecast
test_aligned = test_all.loc[forecast_test_df.index]

# Metrics
rmse = np.sqrt(mean_squared_error(test_aligned["PCE"], df_forecast_level["PCE"]))
rel_rmse = rmse / test_aligned['PCE'].mean()
mape = (np.abs(test_aligned['PCE'] - df_forecast_level['PCE']) / test_aligned['PCE']).mean()

print(f"Relative RMSE: {rel_rmse:.4%}")
print(f"MAPE: {mape:.4%}")

# Plot
plt.figure(figsize=(12,6))
plt.plot(df_forecast_level.index, df_forecast_level['PCE'], label='Forecast', linestyle='--', color='blue')
plt.plot(test_aligned.index, test_aligned['PCE'], label='Actual', color='black', linewidth=2)
plt.title("PCE Forecast vs Actual (Levels)")
plt.xlabel("Date")
plt.ylabel("PCE")
plt.legend()
plt.savefig("forecast_PCE.png", dpi=150, bbox_inches='tight')
plt.close()



Relative RMSE: 1.5419%
MAPE: 1.4371%


In [42]:
print(forecast_test_df['PCE'][0])
print(test['PCE'][0])

-0.0038493263197153793
0.0042754380688325


/var/folders/0s/cw7q4t4101d4kfhgpnb3p61h0000gn/T/ipykernel_81438/3168644159.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(forecast_test_df['PCE'][0])
/var/folders/0s/cw7q4t4101d4kfhgpnb3p61h0000gn/T/ipykernel_81438/3168644159.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(test['PCE'][0])


Saving results

In [43]:
import pickle
import os 
model = VAR(endog=train[cols], exog=train[exog_vars])
results = model.fit(maxlags=9, trend='ctt')
model_dir="models"
os.makedirs(model_dir, exist_ok=True)
with open(f"{model_dir}/var_model_lag9.pkl", "wb") as f:
    pickle.dump(results, f)

/opt/homebrew/Caskroom/miniconda/base/envs/dsc_env/lib/python3.14/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)


In [29]:
results_dir="results"
os.makedirs(results_dir, exist_ok=True)
forecast_test_df.to_csv(f"{results_dir}/forecast_test_stationary.csv")
df_forecast_level.to_csv(f"{results_dir}forecast_level.csv")
metrics = {
    "RMSE": rmse,
    "Relative_RMSE": rel_rmse,
    "MAPE": mape,
    "Selected_Lag": results.k_ar,
}
pd.Series(metrics).to_csv(f"{results_dir}var_metrics.csv")

Residuals Analysis

In [30]:
from statsmodels.stats.diagnostic import acorr_ljungbox

In [31]:

residuals = test_aligned['PCE'] - df_forecast_level['PCE']

plt.figure(figsize=(10,5))
plt.plot(test_aligned.index, residuals, color='red', linewidth=1.5)
plt.axhline(0, color='black', linestyle='--')
plt.title("Residuals (Actual - Forecast)")
plt.xlabel("Date")
plt.ylabel("Residual")
plt.show()

# Ljung-Box: adjust lags to series length
n_obs = len(residuals)
lb_lags = np.arange(3, min(12, n_obs-1)+1)  # safe max lag
lb_test = acorr_ljungbox(residuals, lags=lb_lags, return_df=True)
print("\nLjung-Box p-values (no autocorrelation null):")
print(lb_test)

# Residual histogram
plt.figure(figsize=(6,4))
plt.hist(residuals, bins=15, color='lightblue', edgecolor='black')
plt.title("Distribution of Forecast Residuals")
plt.xlabel("Residual")
plt.ylabel("Frequency")
plt.show()



Ljung-Box p-values (no autocorrelation null):
      lb_stat  lb_pvalue
3   12.626725   0.005517
4   12.627071   0.013249
5   13.601897   0.018346
6   16.312481   0.012171
7   20.891819   0.003934
8   27.491637   0.000581
9   35.803128   0.000043
10  38.925432   0.000026
11  39.232655   0.000048


/var/folders/0s/cw7q4t4101d4kfhgpnb3p61h0000gn/T/ipykernel_81438/1298452820.py:30: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/0s/cw7q4t4101d4kfhgpnb3p61h0000gn/T/ipykernel_81438/1298452820.py:45: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
